In [87]:
import pandas as pd
df= pd.read_csv('/content/DFORE129.csv')

In [88]:
df.client_account_id.unique()

array([1521203,   69966, 1520531,   54628,   54659])

In [89]:
df['flag']= df.client_account_id.map({69966: 'control'}).fillna('test')

In [90]:
df['payment_channel']= df.payment_type.map({'CreditCard-SR': 'MicroTxn', 'Apple-Pay': 'MicroTxn','Google-Pay': 'MicroTxn', 'Coin': 'Coin', 'Stored-Value': 'CSCGo App'})

In [91]:
df.user_id.fillna(df.card_number_last_four, inplace=True)

In [92]:
df.loc[df.payment_type== 'Stored-Value', 'card_number_last_four']

4      NaN
9      NaN
14     NaN
19     NaN
24     NaN
        ..
1492   NaN
1494   NaN
1496   NaN
1498   NaN
1500   NaN
Name: card_number_last_four, Length: 576, dtype: float64

In [93]:
df.user_id.isnull().sum()

2103

In [94]:
df.payment_channel.value_counts(dropna=False)

Coin         1877
CSCGo App     576
MicroTxn      349
Name: payment_channel, dtype: int64

In [95]:
df.groupby('payment_channel')['user_id'].nunique()

payment_channel
CSCGo App    14
Coin          0
MicroTxn     27
Name: user_id, dtype: int64

In [96]:
df['event_timestamp']= pd.to_datetime(df.event_timestamp)

In [97]:
df['week']=df.event_timestamp.dt.year*100+ df.event_timestamp.dt.week

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """Entry point for launching an IPython kernel.


In [98]:
df['month']=df.event_timestamp.dt.year*100+ df.event_timestamp.dt.month

In [99]:
df['year']=df.event_timestamp.dt.year*100+ df.event_timestamp.dt.year

In [100]:
df.columns

Index(['vendor', 'transaction_id', 'correlation_id', 'machine_id', 'room_id',
       'kiosk_id', 'vendor_location_id', 'client_account_id', 'branch_id',
       'machine_type', 'use_type', 'cycle_type', 'amount', 'bonus_amount',
       'credit_amount', 'user_category', 'source', 'payment_type',
       'payment_details', 'consumer_account_id', 'user_id', 'result',
       'failure_reason', 'trans_details', 'card_number_last_four', 'card_type',
       'provider_details', 'payment_gateway', 'mobile_device_info',
       'machine_started', 'trans_updated_at', 'event_timestamp',
       'event_received_timestamp', 'created_at', 'updated_at', 'closed',
       'closed_udm', 'vend_amount', 'transaction_fee_amount', 'flag',
       'payment_channel', 'week', 'month', 'year'],
      dtype='object')

In [101]:
met_dict= {'user_id': 'nunique', 'transaction_id': 'count', 'amount': 'sum'}

In [102]:
grpbyWeek= ['flag', 'branch_id', 'client_account_id', 'week', 'payment_channel']
grpbyMonth= ['flag', 'branch_id', 'client_account_id', 'month', 'payment_channel']
groupbyYear= ['flag', 'branch_id', 'client_account_id', 'year', 'payment_channel']

In [103]:
df1= df.groupby(grpbyCols).agg(met_dict).reset_index().rename(columns={'week':'TimePeriod'})
df1['rec_type']='Week'
df2= df.groupby(grpbyMonth).agg(met_dict).reset_index().rename(columns={'month':'TimePeriod'})
df2['rec_type']='Month'
df3= df.groupby(groupbyYear).agg(met_dict).reset_index().rename(columns={'year':'TimePeriod'})
df3['rec_type']='90 days- LTD'

In [104]:
dfs= [df1,df2,df3]
df4= pd.concat(dfs, axis=0)

In [106]:
df4.rename(columns={'user_id': 'users', 'transaction_id': 'vends',	'amount': 'vendAmt'}, inplace=True)

In [118]:
df5=pd.pivot(df4, index=['rec_type', 'flag', 'branch_id', 'client_account_id', 'TimePeriod'], columns='payment_channel', values= ['users', 'vends', 'vendAmt'])

In [119]:
df5.columns= ['_'.join(col) for col in df5.columns]

In [122]:
df5=df5.reset_index()

In [123]:
df5.columns

Index(['rec_type', 'flag', 'branch_id', 'client_account_id', 'TimePeriod',
       'users_CSCGo App', 'users_Coin', 'users_MicroTxn', 'vends_CSCGo App',
       'vends_Coin', 'vends_MicroTxn', 'vendAmt_CSCGo App', 'vendAmt_Coin',
       'vendAmt_MicroTxn'],
      dtype='object')

In [124]:
df5.to_csv('DFORE129.csv', index=False)